In [12]:
from sqlalchemy import create_engine
import pandas as pd

user = 'root'
password = 'nsn37913148'
host = 'localhost'
port = '3306'
database = 'sakila'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')


In [13]:
def rentals_month(engine, month, year):
    query = f"""
        SELECT rental_id, rental_date, customer_id
        FROM rental
        WHERE MONTH(rental_date) = {month}
        AND YEAR(rental_date) = {year};
    """
    df = pd.read_sql(query, engine)
    return df


In [14]:
def rental_count_month(df, month, year):
    rentals = df.groupby('customer_id').size().reset_index(name=f"rentals_{month:02d}_{year}")
    return rentals

In [15]:
def compare_rentals(df1, df2):
    df = pd.merge(df1, df2, on='customer_id', how='inner')  
    cols = df.columns.tolist()
    df['difference'] = df[cols[2]] - df[cols[1]]  
    return df

In [16]:
may_df = rentals_month(engine, 5, 2005)
june_df = rentals_month(engine, 6, 2005)

may_counts = rental_count_month(may_df, 5, 2005)
june_counts = rental_count_month(june_df, 6, 2005)

comparison = compare_rentals(may_counts, june_counts)

print(comparison.head())


   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7           5
1            2                1                1           0
2            3                2                4           2
3            5                3                5           2
4            6                3                4           1
